In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
from PIL import Image
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import glob
import os
import seaborn as sns
import scipy
import glob
import sklearn 
from tqdm import tqdm
from sklearn.metrics import pairwise_distances
from sklearn.cluster import AffinityPropagation
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.utils import shuffle

In [ ]:
# Latent Space 1
folder_name_1 = 'AE_Training_1'
file_name_1 = '18_Latent_Space'

# Latent Space 2
folder_name_2 = 'AE_Training_2'
file_name_2 = '15_Latent_Space'

# Latent Space 3
folder_name_3 = 'AE_Training_alt'
file_name_3 = '12_Latent_Space'

# Latent 1

In [ ]:
# read in and plot the original data set
latent = pd.read_pickle(f'{folder_name_1}/{file_name_1}')

In [ ]:
print(latent)

In [ ]:
colors = np.array(['blue'] * 40000 + ['red'] * 40000)

plt.figure(figsize=(18,12))
plt.scatter(latent.x, latent.y, c=colors, s=1, alpha=1)
plt.title('Filtered epoch set = 20', fontsize=20)
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.show()

# Latent 2

In [ ]:
# read in and plot the original data set
latent_2 = pd.read_pickle(f'{folder_name_2}/{file_name_2}')

In [ ]:
print(latent_2)

In [ ]:
colors = np.array(['blue'] * 40000 + ['red'] * 40000)

plt.figure(figsize=(18,12))
plt.scatter(latent_2.x, latent_2.y, c=colors, s=1, alpha=1)
plt.title('Filtered epoch set = 20', fontsize=20)
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.show()

# Latent 3

In [ ]:
# read in and plot the original data set
latent_3 = pd.read_pickle(f'{folder_name_3}/{file_name_3}')

In [ ]:
print(latent_3)

In [ ]:
colors = np.array(['blue'] * 40000 + ['red'] * 40000)

plt.figure(figsize=(18,12))
plt.scatter(latent_2.x, latent_2.y, c=colors, s=1, alpha=1)
plt.title('Filtered epoch set = 20', fontsize=20)
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.show()

# Clustering

In [ ]:
# Extract the x and y coordinates for clustering
latent_coords = latent[['x', 'y']].values
latent_2_coords = latent_2[['x', 'y']].values
latent_3_coords = latent_3[['x', 'y']].values

# Normalize the data
scaler = StandardScaler()
latent_coords_scaled = scaler.fit_transform(latent_coords)
latent_2_coords_scaled = scaler.fit_transform(latent_2_coords)
latent_3_coords_scaled = scaler.fit_transform(latent_3_coords)

# Agglomerative clustering on all latent spaces
linkage_matrix1 = linkage(latent_coords_scaled, method='ward')
linkage_matrix2 = linkage(latent_2_coords_scaled, method='ward')
linkage_matrix3 = linkage(latent_3_coords_scaled, method='ward')

# Threshold distance for forming flat clusters
threshold = 1.0  

# Form flat clusters based on the threshold
clusters1 = fcluster(linkage_matrix1, t=threshold, criterion='distance')
clusters2 = fcluster(linkage_matrix2, t=threshold, criterion='distance')
clusters3 = fcluster(linkage_matrix3, t=threshold, criterion='distance')

# Combine the cluster assignments from all latent spaces
combined_clusters = np.vstack((clusters1, clusters2, clusters3)).T

# Find consensus clusters
consensus_clusters = {}
for i, (c1, c2, c3) in enumerate(combined_clusters):
    if (c1, c2, c3) not in consensus_clusters:
        consensus_clusters[(c1, c2, c3)] = []
    consensus_clusters[(c1, c2, c3)].append(int(latent.iloc[i, 0]))

# Filter out singleton clusters (clusters with only one member)
consensus_clusters = {k: v for k, v in consensus_clusters.items() if len(v) > 10}

# Calculate cluster sizes for plotting
cluster_sizes = {k: len(v) for k, v in consensus_clusters.items()}

# Visualization function
def plot_clusters(latent, clusters, cluster_sizes, title):
    cluster_count = pd.Series(clusters).value_counts()
    sizes = cluster_count.loc[clusters].values
    
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(latent['x'], latent['y'], c=clusters, s=sizes, cmap='viridis', marker='o', alpha=0.6)
    plt.title(title)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.colorbar(scatter, label='Cluster')
    plt.show()

# Add cluster labels to the DataFrames for plotting
latent['cluster'] = clusters1
latent_2['cluster'] = clusters2
latent_3['cluster'] = clusters3

# Plot clusters in each latent space
plot_clusters(latent, latent['cluster'], cluster_sizes, 'Clusters in Latent Space 1')
plot_clusters(latent_2, latent_2['cluster'], cluster_sizes, 'Clusters in Latent Space 2')
plot_clusters(latent_3, latent_3['cluster'], cluster_sizes, 'Clusters in Latent Space 3')

# Batched Clustering:

In [ ]:
# Group the DataFrame by 'cluster' and print the indices sorted by the cluster
clustered_indices = latent.groupby('cluster').apply(lambda df: df.index.tolist())

# Print the indices sorted by the cluster
for cluster_id, indices in clustered_indices.items():
    print(f"Cluster {cluster_id}: {indices}")

In [ ]:
# Group the DataFrame by 'cluster' and count the number of indices in each cluster
cluster_counts = latent['cluster'].value_counts().sort_values()

# Print the clusters and their counts sorted by the number of indices in ascending order
for cluster_id, count in cluster_counts.items():
    print(f"Cluster {cluster_id}: {count}")

In [ ]:
print(latent)

# Overlaying Cluster Conformations

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm

# Load the topology and trajectory files for both MD simulations
u1 = mda.Universe("myc_091-160_stripped.prmtop", "aMD_091-160_no1-4_stripped.nc")
u2 = mda.Universe("myc_091-160_D132-H_stripped.prmtop", "aMD_091-160_D132-H_no1-4_stripped.nc")

# Fixed Conformation Display:
- Minimum Opacity (0.002 to prevent blank clusters)

# All Residues in Blue

In [ ]:
# Define the function to extract frames from both universes
def extract_frames(indices, universe1, universe2, switch_index=40000):
    frames = []
    for idx in tqdm(indices, desc="Extracting frames"):
        if idx < switch_index:
            universe1.trajectory[idx]
            frames.append(universe1.select_atoms("name CA").positions.copy())
        else:
            universe2.trajectory[idx - switch_index]
            frames.append(universe2.select_atoms("name CA").positions.copy())
    return frames

# Create a temporary Universe for alignment
def create_temp_universe(positions, universe):
    temp_universe = mda.Merge(universe.select_atoms("name CA"))
    temp_universe.atoms.positions = positions
    return temp_universe

# Align frames using AlignTraj
def align_frames(reference_universe, frames, select_for_alignment):
    aligned_frames = []
    for frame_positions in tqdm(frames, desc="Aligning frames"):
        temp_universe = create_temp_universe(frame_positions, reference_universe)
        aligner = align.AlignTraj(temp_universe, reference_universe, select=select_for_alignment, in_memory=True)
        aligner.run()
        aligned_frames.append(temp_universe.atoms.positions.copy())
    return aligned_frames

# Selection string for residues 15-30 to ensure alignment of corresponding residues
select_residues_15_30 = "resid 15-30 and name CA"

# Function to plot aligned frames for a given cluster
def plot_aligned_frames(cluster_id, indices):
    # Extract frames for the cluster
    frames = extract_frames(indices, u1, u2)
    
    # Create reference universe for alignment
    reference_universe = create_temp_universe(frames[0], u1)
    
    # Align frames using the specific residues 15-30
    aligned_frames = align_frames(reference_universe, frames, select_residues_15_30)
    
    # Set the opacity value with a minimum threshold
    opacity = max(1 / len(aligned_frames), 0.002)
    
    # Plot the aligned frames from different angles
    angles = [(30, 30)]
    
    for angle in angles:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        
        # Plot cluster frames with lines connecting the alpha carbons
        for positions in aligned_frames:
            for i in range(len(positions) - 1):
                color = 'blue'
                ax.plot(positions[i:i+2, 0], positions[i:i+2, 1], positions[i:i+2, 2], 
                        'o-', color=color, markersize=1, markeredgewidth=0.8, alpha=opacity)
        
        ax.view_init(elev=angle[0], azim=angle[1])
        ax.set_axis_off()  # Remove axes
        ax.grid(False)     # Remove grid lines
        plt.title(f'Cluster {cluster_id} Overlap ({len(aligned_frames)} Frames)')
        plt.show()

# Group the DataFrame by 'cluster' and get the indices for each cluster
cluster_indices = latent.groupby('cluster').apply(lambda df: df.index.tolist()).to_dict()

# Plot aligned frames for each cluster
for cluster_id, indices in cluster_indices.items():
    plot_aligned_frames(cluster_id, indices)

# Residue 105 in Red, Rest Blue

In [ ]:
# Define the function to extract frames from both universes
def extract_frames(indices, universe1, universe2, switch_index=40000):
    frames = []
    for idx in tqdm(indices, desc="Extracting frames"):
        if idx < switch_index:
            universe1.trajectory[idx]
            frames.append(universe1.select_atoms("name CA").positions.copy())
        else:
            universe2.trajectory[idx - switch_index]
            frames.append(universe2.select_atoms("name CA").positions.copy())
    return frames

# Create a temporary Universe for alignment
def create_temp_universe(positions, universe):
    temp_universe = mda.Merge(universe.select_atoms("name CA"))
    temp_universe.atoms.positions = positions
    return temp_universe

# Align frames using AlignTraj
def align_frames(reference_universe, frames, select_for_alignment):
    aligned_frames = []
    for frame_positions in tqdm(frames, desc="Aligning frames"):
        temp_universe = create_temp_universe(frame_positions, reference_universe)
        aligner = align.AlignTraj(temp_universe, reference_universe, select=select_for_alignment, in_memory=True)
        aligner.run()
        aligned_frames.append(temp_universe.atoms.positions.copy())
    return aligned_frames

# Selection string for residues 15-30 to ensure alignment of corresponding residues
select_residues_15_30 = "resid 15-30 and name CA"

# Function to plot aligned frames for a given cluster
def plot_aligned_frames(cluster_id, indices):
    # Extract frames for the cluster
    frames = extract_frames(indices, u1, u2)
    
    # Create reference universe for alignment
    reference_universe = create_temp_universe(frames[0], u1)
    
    # Align frames using the specific residues 15-30
    aligned_frames = align_frames(reference_universe, frames, select_residues_15_30)
    
    # Set the opacity value with a minimum threshold
    opacity = max(1 / len(aligned_frames), 0.002)
    
    # Plot the aligned frames from different angles
    angles = [(30, 30)]
    
    for angle in angles:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        
        # Plot cluster frames with lines connecting the alpha carbons
        for positions in aligned_frames:
            for i in range(len(positions) - 1):
                color = 'red' if i == 14 else 'blue'  # 14 is the index for alpha carbon 15
                ax.plot(positions[i:i+2, 0], positions[i:i+2, 1], positions[i:i+2, 2], 
                        'o-', color=color, markersize=1, markeredgewidth=0.8, alpha=opacity)
        
        ax.view_init(elev=angle[0], azim=angle[1])
        ax.set_axis_off()  # Remove axes
        ax.grid(False)     # Remove grid lines
        plt.title(f'Cluster {cluster_id} Overlap ({len(aligned_frames)} Frames)')
        plt.show()

# Group the DataFrame by 'cluster' and get the indices for each cluster
cluster_indices = latent.groupby('cluster').apply(lambda df: df.index.tolist()).to_dict()

# Plot aligned frames for each cluster
for cluster_id, indices in cluster_indices.items():
    plot_aligned_frames(cluster_id, indices)

# Residue 105 in Black, Residue 120 in Blue, Rest in Red

In [ ]:
# Define the function to extract frames from both universes
def extract_frames(indices, universe1, universe2, switch_index=40000):
    frames = []
    for idx in tqdm(indices, desc="Extracting frames"):
        if idx < switch_index:
            universe1.trajectory[idx]
            frames.append(universe1.select_atoms("name CA").positions.copy())
        else:
            universe2.trajectory[idx - switch_index]
            frames.append(universe2.select_atoms("name CA").positions.copy())
    return frames

# Create a temporary Universe for alignment
def create_temp_universe(positions, universe):
    temp_universe = mda.Merge(universe.select_atoms("name CA"))
    temp_universe.atoms.positions = positions
    return temp_universe

# Align frames using AlignTraj
def align_frames(reference_universe, frames, select_for_alignment):
    aligned_frames = []
    for frame_positions in tqdm(frames, desc="Aligning frames"):
        temp_universe = create_temp_universe(frame_positions, reference_universe)
        aligner = align.AlignTraj(temp_universe, reference_universe, select=select_for_alignment, in_memory=True)
        aligner.run()
        aligned_frames.append(temp_universe.atoms.positions.copy())
    return aligned_frames

# Selection string for residues 15-30 to ensure alignment of corresponding residues
select_residues_15_30 = "resid 15-30 and name CA"

# Function to plot aligned frames for a given cluster
def plot_aligned_frames(cluster_id, indices):
    # Extract frames for the cluster
    frames = extract_frames(indices, u1, u2)
    
    # Create reference universe for alignment
    reference_universe = create_temp_universe(frames[0], u1)
    
    # Align frames using the specific residues 15-30
    aligned_frames = align_frames(reference_universe, frames, select_residues_15_30)
    
    # Set the opacity value with a minimum threshold
    opacity = max(1 / len(aligned_frames), 0.002)
    
    # Plot the aligned frames from different angles
    angles = [(30, 30)]
    
    for angle in angles:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        
        # Plot cluster frames with lines connecting the alpha carbons
        for positions in aligned_frames:
            for i in range(len(positions) - 1):
                if i == 14:
                    color = 'black'  # alpha carbon 15
                elif i == 29:
                    color = 'blue'  # alpha carbon 30
                else:
                    color = 'red'
                ax.plot(positions[i:i+2, 0], positions[i:i+2, 1], positions[i:i+2, 2], 
                        'o-', color=color, markersize=1, markeredgewidth=0.8, alpha=opacity)
        
        ax.view_init(elev=angle[0], azim=angle[1])
        ax.set_axis_off()  # Remove axes
        ax.grid(False)     # Remove grid lines
        plt.title(f'Cluster {cluster_id} Overlap ({len(aligned_frames)} Frames)')
        plt.show()

# Group the DataFrame by 'cluster' and get the indices for each cluster
cluster_indices = latent.groupby('cluster').apply(lambda df: df.index.tolist()).to_dict()

# Plot aligned frames for each cluster
for cluster_id, indices in cluster_indices.items():
    plot_aligned_frames(cluster_id, indices)

# Further Analysis:
- Cluster Lifespan
- Transition Preference

In [ ]:
# Step 1: Sort the dataframe by index
latent_sorted = latent.sort_index()

# Step 2: Initialize variables
cluster_durations = []
current_cluster = None
current_duration = 0

# Step 3: Iterate through the sorted dataframe to count consecutive frames in each cluster
for idx, row in latent_sorted.iterrows():
    cluster = row['cluster']
    if cluster == current_cluster:
        current_duration += 1
    else:
        if current_cluster is not None:
            cluster_durations.append((current_cluster, current_duration))
        current_cluster = cluster
        current_duration = 1

# Append the last cluster duration
if current_cluster is not None:
    cluster_durations.append((current_cluster, current_duration))

# Convert the list of durations to a dataframe for easier analysis
durations_df = pd.DataFrame(cluster_durations, columns=['Cluster', 'Duration'])

# Display the dataframe
print(durations_df)